## Tensorboard

Tensorflow에서는 복잡한 네트워크의 동작 상황이나 cost 등 수치의 변화를 관찰하며 디버깅을 할 수 있도록 Tensorboard라는 웹 UI를 지원한다. 아래와 같이 변수에 이름을 붙여주고 그룹을 나눠주면 Tensorboard에서 편리하게 사용할 수 있다.

In [1]:
import numpy as np
import tensorflow as tf

xy = np.loadtxt('train.txt', unpack=True)

x_data = np.transpose(xy[0:-1])
y_data = np.reshape(xy[-1],(4,1))

X = tf.placeholder(tf.float32, name='X-input')
Y = tf.placeholder(tf.float32, name='Y-input')

# define 2 layer Neural Network - we will use 3 layers.
W1 = tf.Variable(tf.random_uniform([2,10], -1.0, 1.0), name="W1")
W2 = tf.Variable(tf.random_uniform([10,10], -1.0, 1.0), name="W2")
W3 = tf.Variable(tf.random_uniform([10,10], -1.0, 1.0), name="W3")
W4 = tf.Variable(tf.random_uniform([10,1], -1.0, 1.0), name="W4")

b1 = tf.Variable(tf.zeros([10]), name="Bias1")
b2 = tf.Variable(tf.zeros([10]), name="Bias2")
b3 = tf.Variable(tf.zeros([10]), name="Bias3")
b4 = tf.Variable(tf.zeros([1]), name="Bias4")

# Draw histograms
w1_hist = tf.summary.histogram("weight1", W1)
w2_hist = tf.summary.histogram("weight2", W2)
w3_hist = tf.summary.histogram("weight3", W3)
w4_hist = tf.summary.histogram("weight4", W4)

b1_hist = tf.summary.histogram("biases1", b1)
b2_hist = tf.summary.histogram("biases2", b2)
b3_hist = tf.summary.histogram("biases3", b3)
b4_hist = tf.summary.histogram("biases4", b4)

y_hist = tf.summary.histogram("y", Y)

# Our hypothesis
with tf.name_scope("layer2") as scope:
    L2 = tf.sigmoid(tf.matmul(X, W1) + b1)

with tf.name_scope("layer3") as scope:
    L3 = tf.sigmoid(tf.matmul(L2, W2) + b2)

with tf.name_scope("layer4") as scope:
    L4 = tf.sigmoid(tf.matmul(L3, W3) + b2)
    
with tf.name_scope("layer5") as scope:
    hypothesis = tf.sigmoid(tf.matmul(L4, W4) + b4)
    
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))
    cost_sum = tf.summary.scalar("cost", cost)
        
# Cross entropy cost function
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(0.3)
    train = optimizer.minimize(cost)

with tf.Session() as sess:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("logs/xor_logs" , sess.graph)
      
    tf.global_variables_initializer().run()
    
    for step in range(40001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 2000 == 0:
            summary, _ = sess.run( [merged, train], feed_dict={X:x_data, Y:y_data})
            writer.add_summary(summary, step)
            
    correct = tf.equal( tf.floor(hypothesis+0.5), Y)
    accuracy = tf.reduce_mean(tf.cast( correct, "float" ) )
    
    # Check accuracy
    print( sess.run( [hypothesis, tf.floor(hypothesis+0.5), correct, accuracy], 
                   feed_dict={X:x_data, Y:y_data}) )
    print( "Accuracy:", accuracy.eval({X:x_data, Y:y_data}) )
    

[array([[  4.49579493e-05],
       [  9.99918222e-01],
       [  9.99936581e-01],
       [  8.82414679e-05]], dtype=float32), array([[ 0.],
       [ 1.],
       [ 1.],
       [ 0.]], dtype=float32), array([[ True],
       [ True],
       [ True],
       [ True]], dtype=bool), 1.0]
Accuracy: 1.0
